# Modelos Lineales Generalizados

> __Alcances de la lectura__:
* Conocer los componentes del marco analítico de los Modelos Lineales Generalizados (Componentes estocásticos, sistemáticos y funciones de enlace).
* Conocer el método de Estimación por Máxima Verosimilitud por el cual se estiman los Modelos Lineales Generalizados.
* Identificar la correcta implementación de los modelos en base a la naturaleza del problema.
* Implementar modelos mediante la librería statsmodels acorde a la naturaleza del problema.
* Interpretar las estimaciones de manera correcta tomando en cuenta las funciones de enlace asociadas a cada modelo.

* Los modelos lineales generalizados responden a un marco analítico para implementar aproximaciones estadísticas a distintos fenómenos manifestados en nuestros vectores objetivos. Resulta que los modelos lineales y logísticos son casos específicos de este marco analítico, donde el predictor estimado es lineal.
* Un modelo lineal generalizado se compone de tres partes: 

    - __Componente aleatorio (o estocástico)__: El componente estocástico de una MLG identifica la familia distribucional que modelará el fenómeno en estudio. Se hace necesario entonces qué problema está resolviendo el vector objetivo Y de manera de ajustar una distribución de probabilidad, normalmente de alguna familia exponencial, para modelar el fenómeno de la manera más fidedigna posible.
    
    - __Componente sistemático__: El componente sistemático especifica las variables exógenas. Siguiendo el nombre del marco analítico, se ingresan de forma lineal-aditiva en el lado derecho de la ecuación. Esta combinación de variables exógenas se conoce como el _predictor lineal_ del modelo ($\mathbf{X}_{i}\beta$).
    
    - __Función de Enlace__: Mediante el componente sistemático podemos aproximarnos al valor esperado de nuestro vector objetivo $\mu=\mathbb{E}(Y)$. Para ello debemos establecer una función vinculante ($g(\cdot)$) entre ambos componentes de manera
    
    $$
    g(\mu) = \mathbf{X}_{i}\beta
    $$


## Miembros de la Familia Exponencial

* Resulta que la mayoría de los componentes aleatorios pertenecen a la familia exponencial. Éstas son un grupo de distribuciones que presentan una serie de propiedades algebráicas generalizables que permiten asimilar de manera relativamente natural los fenómenos.

* Las distribuciones de la familia exponencial presentan la propiedad de _suficiencia estadística_, que asegura que ningún otro estadístico que puede estimarse sobre la misma muestra proporcional información adicional sobre su valor. Esto asegura estabilidad en las estimaciones.

| Distribución | Link Canónico $\theta(\mu)$ | Función Cumulativa $b(\theta)$ | Parámetro de dispersión $\phi$ | Expectativa $\mathbb{E}=b'(\theta)$ | Varianza $\phi b''(\theta)$ |
|---|---|---|---|---|---|
|Normal| $\mu$ | $\theta^{2}/2$ | $\sigma^{2}$ | $\theta$ | $\sigma^{2}$ |
|Bernoulli | $\textsf{ln}(\mu/(1-\mu))$ | $\textsf{ln}(1 + \textsf{exp}(\theta))$ | $1$ | $\frac{\textsf{exp}(\theta)}{1 + \textsf{exp}(\theta)}$ | $\mu(1-\mu)$ |
|Binomial | $\textsf{ln}(\mu/(1-\mu))$ | $\textsf{ln}(1 + \textsf{exp}(\theta))$ | $1/n$ | $\frac{\textsf{exp}(\theta)}{1 + \textsf{exp}(\theta)}$ | $\mu(1-\mu)/n$ |
|Poisson | $\text{ln}(\mu)$ | $\textsf{exp}(\theta)$ | $1$ | $\textsf{exp}(\theta)$ | $\mu$ | 
| Gamma | $-1/\mu$ | $-\textsf{ln}(-\theta)$ | $\alpha^{-1}$ | $-1/\theta$ | $\mu^{2}\alpha^{-1}$ | 
| Gausiana Inversa | $1/\mu^{2}$ | $-(-2\theta)^{1/2}$ | $\sigma^{2}$ | $-(-2\theta)^{1/2}$ | $\mu^{3}\sigma^{2}$ |






## Funciones de enlace

* Para bajar la complejidad de las distribuciones y que tengan sentido dentro de la combinación lineal de los parámetros, implementamos funciones de enlace que permitan esta conversión de los valores. 


| Enlace |Funcíon $g(\mu)$ |Función $g^{-1}(\nu)$ | Rango de  $g^{-1}(\nu)$ | Asociación Distributiva | 
|--------|----------|---------------|-------------------------|---|
|Identidad| $\mu$ | $\nu$ | $-\infty, \infty$ | Normal |
|Recíproco | $1/\mu$ | $1/\nu$ | $-\infty, \infty$ | Normal |
|Logaritmo | $\textsf{ln}(\mu)$ | $\textsf{exp}(\mu)$ | $0, \infty$ | Normal, Poisson, Gamma |
|Logit | $\textsf{ln}\Big(\frac{\mu}{1 - \mu }\Big)$ | $\frac{\textsf{exp}(-\nu)}{1 + \textsf{exp}(\nu)}$ | $0, 1$ | Binomial, Logística-Binomial, Binomial-Beta |
|Probit | $\mathbf{\Phi}^{-1}(\mu)$ | $\mathbf{\Phi}(\nu)$ | $0, 1$ | Binomial | 
|Log-Log Complementario | $\textsf{ln}(-\textsf{ln}(1-\mu))$ | $1 - \textsf{exp}(-\textsf{exp}(\nu))$ | $0, 1$ | Binomial |



## Dinámica

* A lo largo de esta sesión implementaremos distintas aproximaciones MLG:
    - Los modelos _Poisson_ se utilizan cuando nuestro fenómeno observado es un conteo. La función $g(\mu)$ que se utiliza para transformar el valor esperado es la logarítmica, para que el predictor linear $X_{i}\beta$ se transforme en un valor predicho positivo $\hat{y}_{i}$.
    - Los modelos _ordinales_ y _multinomiales_ son implementados cuando nuestro fenómeno observado presenta más de 2 categorías. La función $g(\mu)$ implementa la transformación logit o probit para reescalar el predictor lineal $X_{i}\beta$ se transforme en un valor predicho positivo $\hat{y}_{i}$.
    - Los modelos de _supervivencia_ se utilizan cuando deseamos observar las tasas de mantención o decaimiento de un evento respecto al tiempo. Para este caso implementamos un modelo semiparamétrico donde la respuesta se implementa mediante el estimador _Kaplan-Meier_ y posteriormente se modela con predictores lineales $X_{i}\beta$ que se transforman mediante la función logística.
    
> ### Précis: Estimadores de Máxima Verosimilitud
>
>* La estimación de Máxima Verosimilitud sugiere que el estimador que explique de mejor manera los datos de la muestra (aquél que maximice la verosimilitud de ocurrencia) será el mejor estimador fuera de la muestra.
* La principal ventaja de este método es que tiene propiedades asintóticas. en la medida que el tamaño muestral aumenta, el estimador convergerá de manera más rápida al parámetro poblacional. 
* Por lo general para implementar máxima verosimilitud ejecutamos los siguientes pasos:
    - Formulamos una función de verosimilitud para los datos. La función de verosimilitud no es nada más que la probabilidad de observar cierto dato, condicional a los parámetros ($\text{Pr}(\text{Datos} | \theta)$). La forma de los parámetros dependen de nuestros supuestos.
    - Mediante una técnica de optimización encontramos un estimador para el parámetro $\theta$.

In [1]:
### import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf 
df = pd.read_stata("http://data.princeton.edu/wws509/datasets/docvis.dta")

## Modelación de conteos con un modelo Poisson

* Un modelo Poisson se aplica cuando nuestro vector objetivo responde a una cantidad de ocurrencias de un fenómeno. Así, nuestro fenómeno es no-negativo con un mínimo de 0 y un máximo de $+\infty$.

* En el siguiente ejemplo trabajaremos un registro de pacientes y la cantidad de visitas realizadas al doctor. Algunas de las variables registradas en la base de datos son:
    - `educ`: Nivel educacional
    - `docvis`: Cantidad de visitas
    - `age`: Edad
    - `income`: Nivel de Ingreso
    - `female`: Binario para mujeres
    - `black`: Binario para afroamericanos
    - `hispanic`: Binario para descendientes hispánicos
    - `married`: Binario para casados
    - `noreast`: Binarios para el Noreste.
    - `midwest`: Binario para el Mediooeste.
    - `south`: Binario para el Sur
    - `famsize`: Tamaño de la familia del paciente
    - `injury`: Binario para daño en el paciente
    - `private`: Binario para la visita privada del paciente
    - `chronic`: Binario para un paciente crónico

* Generaremos un modelo donde los predictores serán lineales pero serán transformados mediante el enlace logarítmico, lo que permite
$$
y_{i}\sim\textsf{Poisson}(\textsf{exp}(\mathbf{X}_{i}\beta))
$$

### Interpretación de los coeficientes

* Los coeficientes se pueden exponenciar y considerarse como efectos multiplicativos.

* Un coeficiente es la diferencia esperada de $\textsf{log}(y)$ por cada unidad de incremento en $X$. Si exponenciamos el coeficiente $\textsf{e}^{\beta}$, obtendremos el efecto multiplicativo de la combinación lineal de parámetros $\mathbf{X}_{i}\beta$.


In [2]:
# insertar código

- __Log Likelihood__: Verosimilitud del modelo implementado. Se utiliza para calcular la prueba $\chi^{2}$ si todos los coeficientes son simultáneamente 0. También se implementa para comparar la capacidad explicativa del modelo _dentro de la muestra_ cuando se compara con el Log Likelihood de un modelo nulo.
- __LLR p-value__: Es el puntaje de la prueba $\chi^{2}$ de significancia conjunta de los regresores. Si el puntaje es menor a 0.05 estamos habilitados para rechazar la hipótesis nula de insignificancia conjunta de los regresores. En este caso la evidencia sugiere que por lo menos un coeficiente será distinto de 0.
- __LL-Null__: Es la verosimilitud del modelo cuando no se especifican regresores lineales que afecten las chances en una de las categorías. En este caso, el modelo con regresores minimiza la distancia respecto a la verosimilitud en aproximadamente 4557 puntos. Esto sugiere que nuestra combinación de predictores lineales mejora el desempeño del modelo.
 - __Pseudo R-squared.__: Corresponde a un R-squared penalizado dependiendo de la complejidad del modelo, es decir, a mayor número de parámetros predictores mayor es la penalización.

* Consideremos las variables binarias `private` y `chronic`. Entre dos pacientes con características similares que difieren sólo en que uno es un paciente crónico, éste tiene 3 veces más chances de visitar al doctor en comparación al otro individuo.

* Entre dos pacientes con características similares que sólo difieren en que uno asistió a un clínica privada, éste tiene 2 veces más chances de visitar al doctor en comparación al otro individuo.



# Modelando múltiples categorías con una regresión multinomial

* También conocido como un modelo clasificador multiclases, la regresión logística multinomial se implementa cuando tenemos más de dos categorías en nuestro vector objetivo y es difícil discenir cuando existe jerarquía entre ambas. De manera similar a la operacionalización logística binaria, el objeto es ver las chances de ocurrencia en $j \in \mathcal{J}$ evento. La ecuación a estimar queda así:

$$
\text{Pr}(Y_{i} = j) = \frac{\textsf{exp}(X_{i}^{\textsf{T}}\beta_{j})}{1+\sum_{k=1}^{\mathcal{J-1}}\textsf{exp}(X_{i}^{\textsf{T}}\beta_{k})}
$$

donde $X_{i}\beta$ es la combinación lineal de parámetros a estimar. La sumatoria del denominador es válida en el rango $\mathcal{J}-1$, para excluír una categoría que será de referencia (y permitirá hacer el modelo identificable).

* Para este ejemplo utilizaremos una base de datos sobre actitudes vocacionales de 200 estudiantes. Modelaremos mediante una regresión multinomial las chances de proseguir estudios en un programa académico, general o vocacional. Las covariables a implementar como explicativas son:
    - `female`: El sexo del estudiante (`male` o `female`).
    - `ses`: Nivel socioeconómico (`low`, `middle`, `high`).
    - `schtyp`: El tipo de escuela a la que asisten (`public`, `private`).
    - `prog`: El tipo de programa a proseguir.
    - `read`, `write`, `math`, `science`, `socst`: Los puntajes de cada alumno en las asignaturas.
    - `honors`: ¿Está en un programa de honores? (`enrolled`, `not_enrolled`)
    - `awards`: Cantidad de premios recibidos.
    
* Antes de implementar nuestro modelo generaremos recodificaciones en las variables con strings, para facilitar el desarrollo del modelo.

In [3]:
df = pd.read_stata('https://stats.idre.ucla.edu/stat/data/hsbdemo.dta')

In [4]:
#insertar código

* Para implementar el modelo con `statsmodels` ocuparemos el método `mnlogit` que recibe los parámetros `formula` y `df`. 

In [5]:
#insertar código

* De manera similar a como lo hicimos con el modelo Poisson, vamos a revisar las estadísticas de bondad de ajuste del modelo generado, para ello nos fijaremos en 3 características:

- __Log Likelihood__: Verosimilitud del modelo implementado. Se utiliza para calcular la prueba $\chi^{2}$ si todos los coeficientes son simultáneamente 0. También se implementa para comparar la capacidad explicativa del modelo _dentro de la muestra_ cuando se compara con el Log Likelihood de un modelo nulo.
- __LLR p-value__: Es el puntaje de la prueba $\chi^{2}$ de significancia conjunta de los regresores. Si el puntaje es menor a 0.05 estamos habilitados para rechazar la hipótesis nula de insignificancia conjunta de los regresores. En este caso la evidencia sugiere que por lo menos un coeficiente será distinto de 0.
- __LL-Null__: Es la verosimilitud del modelo cuando no se especifican regresores lineales que afecten las chances en una de las categorías. En este caso, el modelo con regresores minimiza la distancia respecto a la verosimilitud en aproximadamente 45 puntos. Esto sugiere que nuestra combinación de predictores lineales mejora el desempeño del modelo.
 - __Pseudo R-squared.__: Corresponde a un R-squared penalizado dependiendo de la complejidad del modelo, es decir, a mayor número de parámetros predictores mayor es la penalización.

* Cuando estimamos un modelo multinomial se define una categoría de referencia en nuestro vector objetivo, de similar manera a cómo lo desarrollamos en un modelo logístico binario. 

* Por defecto, statsmodels define de forma automática cuál va a ser la categoría de referencia. Los coeficientes estimados en estos modelos son el efecto en el logaritmo de chance la probabilidad lineal en la categoría específica.

* Para estos casos podemos realizar interpretaciones súbitas como la regla de dividir por 4 o exponenciar el coeficiente y evaluarlo respecto a 1.

* Partamos por discriminar entre aquellos coeficientes que satisfacen con los criterios arbitrarios de significancia estadística. 

* Cuando analizamos las chances que un alumno tome un programa académico respecto a un programa de formación general, atributos como su desempeño en matemáticas y ciencias, así como el nivel socioeconómico están asociados de manera significativa con mayores chances.

* Respecto a las chances de que un alumno tome programas vocacionales, sólo el nivel socioeconómico está débilmente asociado con mayores chances.

## Cambio marginal en los parámetros estimados

* Dado la complejidad de interpretar los log-odds estimados en los modelos con variables discretas, una opción para entender el efecto de $x$ en $Pr(y)$ es mediante la obtención de la derivada parcial de la pendiente que asocia ambas variables mediante la siguiente ecuación:

$$
\frac{\partial\ \textsf{Pr}(y = j | \mathbf{x})}{\partial x_{k}}
=
\textsf{Pr}(y = j | \mathbf{x})
\Bigg(\beta_{k, m|\mathcal{J}} - \sum_{j=1}^{\mathcal{J}} \beta_{k, j| \mathcal{J}} \textsf{Pr}(y = j | \mathbf{x}) \Bigg)
$$

* Una vez que nuestro objeto creado con `smf.mnlogit` es ejecutado, tiene el método `get_margeff` que calcula los efectos marginales por cada parámetro. Por defecto todos los efectos estimados se generan calculando el promedio de los efectos marginales a lo largo de cada valor de $x_{k}$.
* A diferencia de otras implementaciones de estimación de efectos marginales para modelos multinomiales (`stata` y `R`), esta implementación devuelve los efectos para todas las categorías en el vector objetivo.
* Si solicitamos el resumen, el efecto marginal reportado se encuentra en la columna `dy/dx`. Volviendo a los resultados obtenidos al inspeccionar los log-odds, esperamos que para las chances de seguir un programa académico _el aumentar los puntajes en matemática en 1 unidad conlleva a un aumento de 0.0176 en la probabilidad de éste_. 
* De manera opuesta, _el aumento en 1 unidad de los puntajes de ciencia conlleva a un decrecimiento de 0.0145 en la probabilidad de proseguir con un programa académico_.

In [6]:
#insertar código

# Modelando supervivencia con un modelo de riesgos proporcionales


### Ejemplo: Modelando la tasa de reincidencia carcelaria

* Para este ejemplo utilizaremos datos provinientes de 432 prisioneros que fueron observados a lo largo de un año posterior a ser liberados de la cárcel. Las variables de la base de datos `rossi` son las siguientes:

    * `week`: Semanas del primer arresto desde la liberación.
    * `arrest`: Evento a analizar, 1 si el individuo fue arrestado en el periodo de análisis y 0 de lo contrario.
    * `fin`: Ayuda financiera al individuo posterior a su liberación. "yes" si recibió, "no" de lo contrario.
    * `age`: Edad del individuo.
    * `wexp`: Experiencia laboral previa del individuo. "yes" si tiene, "no" de lo contrario.
    * `race`: Raza del individuo. "black" para afrodescendientes, "other" para latinos, asiático y caucásicos.
    * `mar`: Estado civil del individuo. "married" si estaba casado al momento de ser liberado, "not married" de lo contrario.
    * `paro`: Libertad condicional del individuo al momento de salir. "yes" si lo estaba, "no" de lo contrario.
    * `prio`: Cantidad de encarcelamientos previos del individuo.

## Modelo de Riesgos Proporcionales de Cox

* El modelo de Cox busca examinar la relación la distribución de supervivencia de un evento dada una serie de covariables. El modelo se considera como _semiparamétrico_ dado que genera dos estimaciones, una _no paramétrica_ donde el riesgo basal puede tomar cualquier forma funcional, y una _paramétrica_ que estima el impacto de los predictores __lineales__ del modelo.
* El modelo de Cox considera el intercepto ($\alpha(t)=\text{log}h_{0}(t)$) como indefinido respecto al tiempo (definido como $t$:

$$
\textsf{log} h_{i}(t) = \alpha(t) + \beta_{1}x_{i1}+\beta_{2}x_{i2} + \cdots + \beta_{k}x_{ik}
$$

* Si tomamos dos observaciones $i$ e $i'$ que difieren en 1 unidad en sus $x$, las predicciones lineales estimadas son:

$$
\eta_{i}=\beta_{1}x_{i1}+\beta_{2}x_{i2} + \cdots + \beta_{k}x_{ik}
$$

y

$$
\eta_{i'}'=\beta_{1}x_{i1}+\beta_{2}x_{i2} + \cdots + \beta_{k}x_{ik}
$$

podemos tomar la razón de riesgos de $\eta$ y $\eta '$ como

$$
\begin{eqnarray}
\frac{h_{i}(t)}{h_{i'}(t)} & = \frac{h_{0}(t)\textsf{exp}(\eta_{i})}{h_{0}(t)\textsf{exp}(\eta_{i'})} \\
                           & = \frac{\textsf{exp}(\eta_{i})}{\textsf{exp}(\eta_{i'})} \\
\end{eqnarray}
$$

Esta razón de riesgos es independiente del tiempo y mide el riesgo proporcional entre dos combinaciones de predictores lineales.

## Implementando el modelo con `lifelines`

* Para estimar el modelo utilizaremos la librería `lifelines` que debemos agregar a nuestra carpeta `conda``conda install -c conda-forge lifelines`. Recuerden que utilizamos `conda-forge` porque son librerías externas al proyecto `anaconda`.
* El modelo de riesgos proporcionales se encuentra en la clase `CoxPHFitter`. Dentro de los argumentos de la clase encontramos `alpha`, que determina el nivel de los intervalos de confianza a estimar en el modelo; `tie_method`, que especifica el método a implementar ante el caso de empates entre observaciones. Por el momento sólo está disponiblel el método de `'efron'`. Otros lenguajes como `R` ofrecen más métodos de resolución de empate como `'breslow'` y `'exact'`.

In [7]:
from lifelines.datasets import load_rossi
df = load_rossi()

ModuleNotFoundError: No module named 'lifelines'

In [ ]:
#insertar código

* Una vez que instanciamos nuestro objeto con la clase `CoxPHFitter`, podemos pasar los datos donde generaremos el modelo. Debemos utilizar el método `fit` en nuestro objeto. En su versión más reducida, se necesitan declarar tres argumentos:
    - `df`: La base de datos donde implementaremos el análisis.
    - `duration_col`: La variable que mide la exposición a lo largo del tiempo.
    - `event_col`: El evento a estimar la supervivencia.

* Con nuestro modelo estimado podemos pedir los coeficientes con `print_summary()`. Si aplicamos las prácticas comunes de interpretación, observamos que hay 3 factores estadeisticamente significativos (asumiendo un nivel de significancia nominal del 95%) asociados a la reincidencia carcelaria: la edad del individuo (`age`), la cantidad de encarcelamientos previos (`prio`) y ayuda financiera (`fin`).
* Podemos estar tentados a interpretar los coeficientes de forma lineal, pero los resultados de los coeficientes vienen exponenciados en `exp(coef)`. Estos se pueden interpretar como efectos multiplicativos en el riesgo de reincidencia. Así, observamos que entre dos individuos que difieren en un año de edad hay una disminución en el riesgo _promedio_ de 5.6% en la reincidencia. Este número se estima a partir de $(1 - \text{exp}(\beta_{2}\texttt{age})) = (1 - 0.944) = 5.6$.
* Por lo general, comparamos el coeficiente exponenciado respecto a 1. Si la diferencia entre 1 y $\text{exp}(\beta)$ es positiva, se dice que la covariable aumenta la tasa de supervivencia. De siminar la manera si la diferencia entre 1 y $\text{exp}\beta$, se dice que la covariable disminuye la tasa de supervivencia.
* La cantidad de encarcelamientos está asociado negativamente con el tiempo fuera de la cárcel del individuo. Cada una aumenta el riesgo en 9.6% ($(1 - \text{exp}(\beta_{7}\texttt{prio})) = (1 - 1.096) = 9.6$).

## Visualizando la tasa de reincidencia

* Ya sabemos el efecto de las covariables inspeccionadas. Ahora podemos analizar la evolución de la reincidencia en la muestra completa. Para ello implementaremos el estimador de Kaplan-Meier, un método no paramétrico para estimar la función de supervivencia. El estimador viene de

$$
\hat{S}(t)=\prod_{1: t_{i}\leq t} \Bigg(1 - \frac{d_{i}}{n_{i}}\Bigg)
$$ 

donde se resuelve el número de eventos $d_{i}$ por sobre la cantidad de individuos _sin eventos ocurridos_ en tiempo $t_{i}$. Esto se calcula por cada periodo y posteriormente se multiplican los resultados.

* Por lo general el estimador se visualiza como una serie de peldaños descendientes y mide la disminución de no ocurrencia del evento a lo largo del tiempo.
* Este es el componente noparamétrico del modelo de riesgos proporcionales de Cox.

### Visualizando el efecto de una covariable a lo largo del tiempo

* Hasta el momento hemos modelado el riesgo del evento mediante la combinación lineal de los predictores. Dada la naturaleza semiparamétrica del modelo de riesgos proporcionales, podemos evaluar la evolución del riesgo para ciertos valores en la medida que pasa el tiempo.
* Una vez que ya estimamos el modelo de riesgos proporcionales, el objeto ofrece el método `plot_covariate_group`. Dentro de este método uno debe declarar dos argumentos: 1. la covariable a analizar, la cual debe estar presente en el modelo; 2. los puntajes donde evaluaremos el efecto de la variable a lo largo del tiempo.
* Para este caso analizaremos el efecto de la ayuda financiera en las tasas de encarcelamiento. En este caso el método devuelve tres escenarios: dos por los valores a evaluar y un punto basal que responde cuando nuestra combinación lineal de valores está en 0.
* Observamos que una vez que salen de la cárcel, todos los individuos parten con una probabilidad de no reincidir de 1. En la medida que el tiempo pasa, el riesgo de no reincidir en aquellos individuos sin ayuda financiera disminuye de una forma más acelerada que el grupo con ayuda financiera.

* Para analizar los riesgos en las covariables contínuas, se declara un array con cada valor a evaluar en  `plot_covariate_groups`. Vamos a graficar los riesgos para las otras dos variables "significativas".
* Observamos que cuando los individuos liberados no tienen encarcelamientos previos tienen mayores chances de no reincidir, en comparación a la categoría basal. 

* De similar manera, se observa que individuos tienen mayores chances de reincidir.